# Simple Training with the 🤗 Transformers Trainer


c.f. Lewis Tunstall & HuggingFace 🤗 video on [Simple Training with the 🤗 Transformers Trainer](https://www.youtube.com/watch?v=u--UVvH-LIQ&t=132s)

## Dataset: EDA & preparation

In [1]:
from datasets import load_dataset

emotion_dataset = load_dataset("emotion")
emotion_dataset

Found cached dataset emotion (/home/brooke_fujita/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

What do the dataset items look like?

In [2]:
emotion_dataset["train"][0]

{'text': 'i didnt feel humiliated', 'label': 0}

Sometimes, it is easier to inspect a dataset with Pandas DataFrames.

In [3]:
emotion_df = emotion_dataset["train"].to_pandas()
emotion_df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


----

In [4]:
type(emotion_dataset["train"])

datasets.arrow_dataset.Dataset

In [5]:
features = emotion_dataset["train"].features
features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [6]:
features['label'].int2str(0)

'sadness'

In [7]:
n_classes = features['label'].num_classes

In [8]:
id2label = {idx:features['label'].int2str(idx) for idx in range(n_classes)}
id2label

{0: 'sadness', 1: 'joy', 2: 'love', 3: 'anger', 4: 'fear', 5: 'surprise'}

In [9]:
label2id = {k:v for v,k in id2label.items()}
label2id

{'sadness': 0, 'joy': 1, 'love': 2, 'anger': 3, 'fear': 4, 'surprise': 5}

----

In [10]:
class2prob = emotion_df["label"].value_counts(normalize=True).sort_index()
print(class2prob)

0    0.291625
1    0.335125
2    0.081500
3    0.134937
4    0.121063
5    0.035750
Name: label, dtype: float64


In [11]:
for i,v in class2prob.items():
    print(f"{id2label[i]:<10} {v:.5F}")

sadness    0.29163
joy        0.33513
love       0.08150
anger      0.13494
fear       0.12106
surprise   0.03575


Notice that class "5" for "surprise" is not well represented within the dataset.

How do we deal with this?

## Tokenization

[microsoft/MiniLM-L12-H384-uncased](https://huggingface.co/microsoft/MiniLM-L12-H384-uncased?text=I+like+you.+I+love+you) on Hugging Face modelhub.

> Pre-trained language models (e.g., BERT (Devlin et al., 2018) and its variants) have achieved remarkable success in varieties of NLP tasks. However, these models usually consist of hundreds of millions of parameters which brings challenges for fine-tuning and online serving in real-life applications due to latency and capacity constraints. In this work, we present a simple and effective approach to compress large Transformer (Vaswani et al., 2017) based pre-trained models, termed as deep self-attention distillation. The small model (student) is trained by deeply mimicking the self-attention module, which plays a vital role in Transformer networks, of the large model (teacher). Specifically, we propose distilling the self-attention module of the last Transformer layer of the teacher, which is effective and flexible for the student. Furthermore, we introduce the scaled dot-product between values in the self-attention module as the new deep self-attention knowledge, in addition to the attention distributions (i.e., the scaled dot-product of queries and keys) that have been used in existing works. Moreover, we show that introducing a teacher assistant (Mirzadeh et al., 2019) also helps the distillation of large pre-trained Transformer models. Experimental results demonstrate that our monolingual model outperforms state-of-the-art baselines in different parameter size of student models. In particular, it retains more than 99% accuracy on SQuAD 2.0 and several GLUE benchmark tasks using 50% of the Transformer parameters and computations of the teacher model. We also obtain competitive results in applying deep self-attention distillation to multilingual pre-trained models.

Read the [MiniLM: Deep Self-Attention Distillation for Task-Agnostic Compression of Pre-trained Transformers](https://arxiv.org/abs/2002.10957) paper.

In [13]:
from transformers import AutoTokenizer

checkpoint = "microsoft/MiniLM-L12-H384-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

`input_ids`

> Index ids of the word embeddings corresponding to the given word.

`token_type_ids`

> ... a binary mask identifying the two types of sequence in the model ...
> ... the “context” used for the question, has all its tokens represented by a 0, whereas the second sequence, corresponding to the “question”, has all its tokens represented by a 1...
> Some models, like XLNetModel use an additional token represented by a 2.

`attention_mask`

> Attention masks are tensors with the exact same shape as the input IDs tensor, filled with 0s and 1s: 1s indicate the corresponding tokens should be attended to, and 0s indicate the corresponding tokens should not be attended to (i.e., they should be ignored by the attention layers of the model).

In [14]:
tokenizer(emotion_dataset["train"]["text"][:1])

{'input_ids': [[101, 1045, 2134, 2102, 2514, 26608, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1]]}

In [15]:
def tokenize_text(examples):
    return tokenizer(examples["text"], truncation=True, max_length=512)

In [16]:
emotion_dataset = emotion_dataset.map(tokenize_text, batched=True)
emotion_dataset

Loading cached processed dataset at /home/brooke_fujita/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-7b24961a81fff009.arrow


  0%|          | 0/2 [00:00<?, ?ba/s]

Loading cached processed dataset at /home/brooke_fujita/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705/cache-7425973162d2e3d7.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

From the API docs on Trainer:

> The Trainer class is optimized for 🤗 Transformers models and can have surprising behaviors when you use it on other models. When using it on your own model, make sure:
>
> * your model always return tuples or subclasses of ModelOutput.
> * your model can compute the loss if a labels argument is provided and that loss is returned as the first element of the tuple (if your model returns tuples)
> * your model can accept multiple label arguments (use the label_names in your TrainingArguments to indicate their name to the Trainer) but none of them should be named "label".

So, we need to rename that column for label in our dataset to labels.

In [17]:
emotion_dataset = emotion_dataset.rename_column("label", "labels")
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

----

## Dealing with imbalanced classes

In [18]:
class_weights = (1 - (emotion_df["label"].value_counts().sort_index() / len(emotion_df))).values
print(class_weights)
print(type(class_weights))

[0.708375  0.664875  0.9185    0.8650625 0.8789375 0.96425  ]
<class 'numpy.ndarray'>


In [19]:
import torch

class_weights = torch.from_numpy(class_weights).float().to("cuda")
class_weights
print(type(class_weights))
print(class_weights.dim())

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
<class 'torch.Tensor'>
1


Now, when training a classfier using a dataset that may suffer for imbalanced classes, one way to deal with the situation is to up-sample from the imbalanced class(es) to offset the differnce(s). However, Transformer models are good at memorizing patterns, and so this might not be a good idea in our case.

What we can do is this: correct for the class imbalance with the loss function during training.

From the [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer#trainer) documentation:

> To inject custom behavior you can subclass them and override the following methods: ...
> * compute_loss - Computes the loss on a batch of training inputs.


From PyTorch docs on [`torch.nn.CrossEntropyLoss`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html):

> If provided, the optional argument weight should be a 1D Tensor assigning weight to each of the classes. This is particularly useful when you have an unbalanced training set.

In [20]:
from torch import nn
import torch
from transformers import Trainer


class WeightedLossTrainer(Trainer):
    
    def compute_loss(self, model, inputs, return_outputs=False):
        # feed inputs to model and extract logits
        outputs = model(**inputs)
        logits = outputs.get("logits")
        
        # extract labels; not label!
        #         ^^^^^
        labels = inputs.get("labels")
        
        # define loss function with class weights
        loss_func = nn.CrossEntropyLoss(weight=class_weights)
        
        # compute loss
        loss = loss_func(logits, labels)
        
        return (loss, outputs) if return_outputs else loss

----

## Now bring it all together!

In [21]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels=n_classes,
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
from sklearn.metrics import f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    return {"f1": f1}

In [23]:
from transformers import TrainingArguments

batch_size = 64

logging_steps = len(emotion_dataset["train"]) // batch_size

output_dir = "test-minilm-finetuned-emotion"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_steps=logging_steps,
    fp16=True,
    push_to_hub=False
)

In [24]:
trainer = WeightedLossTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=emotion_dataset["train"],
    eval_dataset=emotion_dataset["validation"],
    tokenizer=tokenizer
)

Using cuda_amp half precision backend


You may see a warning message stating <code>Using cuda_amp half precision backend</code>.

From [Nvidia, Deep Learning Performance Documentation, Train with Mixed Precision](https://docs.nvidia.com/deeplearning/performance/mixed-precision-training/index.html)...
> There are numerous benefits to using numerical formats with lower precision than 32-bit floating point. First, they require less memory, enabling the training and deployment of larger neural networks. Second, they require less memory bandwidth which speeds up data transfer operations. Third, math operations run much faster in reduced precision, especially on GPUs with Tensor Core support for that precision. Mixed precision training achieves all these benefits while ensuring that no task-specific accuracy is lost compared to full precision training. It does so by identifying the steps that require full precision and using 32-bit floating point for only those steps while using 16-bit floating point everywhere else.

c.f. [CUDA Automatic Mixed Precision Examples](https://pytorch.org/docs/stable/notes/amp_examples.html#cuda-automatic-mixed-precision-examples)

In [25]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/opt/conda/envs/transformers-py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16000
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text 

Epoch,Training Loss,Validation Loss,F1
1,1.386400,1.062423,0.577758
2,0.897200,0.713546,0.815931
3,0.621400,0.509548,0.888702
4,0.467100,0.432829,0.904052
5,0.404900,0.395059,0.906486


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 64
Saving model checkpoint to test-minilm-finetuned-emotion/checkpoint-500
Configuration saved in test-minilm-finetuned-emotion/checkpoint-500/config.json
Model weights saved in test-minilm-finetuned-emotion/checkpoint-500/pytorch_model.bin
tokenizer config file saved in test-minilm-finetuned-emotion/checkpoint-500/tokenizer_config.json
Special tokens file saved in test-minilm-finetuned-emotion/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely 

TrainOutput(global_step=1250, training_loss=0.7554116088867188, metrics={'train_runtime': 138.6087, 'train_samples_per_second': 577.164, 'train_steps_per_second': 9.018, 'total_flos': 582587326282752.0, 'train_loss': 0.7554116088867188, 'epoch': 5.0})

----

## Try out our newly-trained model

In [26]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
00_simple_training.ipynb  environment.transformers-py38.yml
01_asr.ipynb		  test-minilm-finetuned-emotion
README.md		  whisper_intro.m4a


In [27]:
from transformers import pipeline

pipl = pipeline("text-classification", 
                model="test-minilm-finetuned-emotion/checkpoint-1000")

loading configuration file test-minilm-finetuned-emotion/checkpoint-1000/config.json
Model config BertConfig {
  "_name_or_path": "test-minilm-finetuned-emotion/checkpoint-1000",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "id2label": {
    "0": "sadness",
    "1": "joy",
    "2": "love",
    "3": "anger",
    "4": "fear",
    "5": "surprise"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "label2id": {
    "anger": 3,
    "fear": 4,
    "joy": 1,
    "love": 2,
    "sadness": 0,
    "surprise": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.2

In [28]:
pipl("I am sorry to see you go")

[{'label': 'sadness', 'score': 0.8894500732421875}]